### NOMIS API Data Extract for Historic Census Data.

The NOMIS RESTful API limits the size of the 'queries / calls' you can request.
For example, you cannot request all of the geography codes in a single call, the site will time-out or error due to the number of data items. Additionally each call is limited to a maximum of 25k records; hence the need for a few nested programming loops etc. etc. etc.

***Here are is a useful reading resource:***
https://www.nomisweb.co.uk/api/v01/help

***Here is the target data Darren Barnes has requested:***

"*Had a chat with census team this morning and this is the dataset they would like us to test out
Its a 2011 census table LC3409EW >> https://www.nomisweb.co.uk/census/2011/lc3409ew and includes General Health, Tenure and Age, covers England and Wales, and goes down to OA level.
obviously its available through the NOMIS API and i would like us to suck that into COGS rather than download and transform. The census team have had some very good feedback from Ahmed on the data viz and sparql and i think this has generated some buzz within the census senior management team. So the idea is to load this data into COGS and have Ahmed do a data viz on it and compare that with the CMD.
We have looked at NOMIS API already and i would like us to give consideration on what and when we could get this loaded into COGS."*

**OA**: Census **O**utput **A**reas.

In [ ]:
# Components & Libraries:
import pandas as pd
import json
from pathlib import Path
import datetime
import numpy as np
from gssutils import pathify

from cachecontrol import CacheControl
from cachecontrol.caches import FileCache
from cachecontrol.heuristics import ExpiresAfter
from requests import Session
session = CacheControl(Session(), cache=FileCache('.cache'), heuristic=ExpiresAfter(days=7))

import re

pd.set_option('display.max_colwidth', -1)

absolute_start = datetime.datetime.now()

In [ ]:

# NM_2010_1
str_dataset_id = "NM_2010_1"

print(str_dataset_id)
print('https://www.nomisweb.co.uk/api/v01/dataset/NM_2010_1/geography.def.sdmx.json')

In [ ]:
# NOMIS RESTful: Get Parent Level Geography for Dataset:

# Get Geography sets using our ID:
baseURL_Geography = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography.def.sdmx.json'
df_Geography = pd.read_json(baseURL_Geography)

# Not sure if we want more than one geographies for this, but sooner or later we will do so 
# we'll pretend that we do for now
geographies_we_want = ["2011 output areas"]
geographies_found = []

for a_code in df_Geography["structure"]['codelists']['codelist'][0]['code']:

    if 'description' not in a_code.keys():
        continue
        
    geog_found_name = a_code['description']['value']
    
    for geography_we_want in geographies_we_want:
        if geog_found_name == geography_we_want:
            # Take the whole code dict, never know what we'll need later
            geographies_found.append(a_code)
                 
# Sanity check
assert len(geographies_found) == len(geographies_we_want), \
    "Aborting, we're missing geographies. Wanted {}, got {}.".format(json.dumps(geographies_we_want, indent=2),
                                                                        json.dumps(geographies_found, indent=2))

df_Geography = pd.DataFrame() # Memory

print("Geographies selected: {}".format(json.dumps(geographies_found, indent=2)))


In [ ]:

# Get all the dependant codes for these geographies
codes_what_we_want = []
for a_parent_code_dict in geographies_found:

    base_geography_url = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography/' + a_parent_code_dict["value"] + '.def.sdmx.json'
    base_geography_dict = session.get(base_geography_url).json()
    
    assert len(base_geography_dict["structure"]["codelists"]) == 1, "We should only have one codelists being" \
                " returned from this call."
    
    for a_child_code_dict in base_geography_dict["structure"]["codelists"]["codelist"][0]["code"]:
        """
        this is one, I'm taking the 9 digit geography code to save us a job later
        -----
        {
        "annotations": {
            "annotation": [
                {
                    "annotationtext": "2011 output areas",
                    "annotationtitle": "TypeName"
                },
                {
                    "annotationtext": 299,
                    "annotationtitle": "TypeCode"
                },
                {
                    "annotationtext": "E00174208",
                    "annotationtitle": "GeogCode"
                }
            ]
        },
        "parentcode": 1228931073,
        "description": {
            "value": "E00174208",
            "lang": "en"
        },
        "value": 1254265842
        },
        """
        try:
            area = a_child_code_dict["annotations"]["annotation"][0]["annotationtext"]
            if area != "2011 output areas":
                raise Exception("This is supposed to be 2011 output areas, got {}.".format(area))
            
            code = a_child_code_dict["description"]["value"]
            #code = a_child_code_dict["value"]
            
            codes_what_we_want.append(code)
        except Exception as e:
            raise Exception("Failed on", json.dumps(a_child_code_dict)) from e

print("Unique codes identified: ", len(set(codes_what_we_want)))

In [ ]:
# Check they seem to be in some sort of order
codes_what_we_want.sort()
print(codes_what_we_want[:5])
print(codes_what_we_want[-5:])

In [ ]:
# Define the fields that we want
fields_we_want = ["DATE","DATE_NAME","DATE_CODE","DATE_TYPE","DATE_TYPECODE","DATE_SORTORDER",
                  "GEOGRAPHY","GEOGRAPHY_NAME","GEOGRAPHY_CODE","GEOGRAPHY_TYPE","GEOGRAPHY_TYPECODE",
                  "GEOGRAPHY_SORTORDER","GENDER","GENDER_NAME","GENDER_CODE","GENDER_TYPE",
                  "GENDER_TYPECODE","GENDER_SORTORDER","C_AGE","C_AGE_NAME","C_AGE_CODE","C_AGE_TYPE",
                  "C_AGE_TYPECODE","C_AGE_SORTORDER","MEASURES","MEASURES_NAME","OBS_VALUE","OBS_STATUS",
                  "OBS_STATUS_NAME","OBS_CONF","OBS_CONF_NAME","URN","RECORD_OFFSET","RECORD_COUNT"]

fields_we_want_query_str = ",".join(fields_we_want)
fields_we_want_query_str

# SubSets

I'm going to just grab a subset of 10 codes otherwise this will end in madness


In [ ]:

# IMPORTANT - hard coding a smaller subset to try it out
codes_what_we_want = [str(x) for x in codes_what_we_want]

count = 0
end = 99999999999999999 # for now
increment = 10
endnextloop = False
got_since_flush = 0
output_counter = 0
final_df = pd.DataFrame()

while True:

    if endnextloop:
        break
        
    #print("Count {}, Increment {}, end {}".format(count, increment, end))
    
    # Get codes by finite increments then finish with whatever the leftovers are
    if count+increment > len(codes_what_we_want):
        start = count
        end = (len(codes_what_we_want)-count)+count
        endnextloop = True
    else:
        start = count
        end = count+increment
        count = end
        endnextloop = False
        
    got_since_flush += increment
        
    codes_to_get = ",".join(codes_what_we_want[start:end])
    if len(codes_to_get) == 0:
        break
        
    #print("Getting codes: ", codes_to_get)
    
    #import sys
    #sys.exit(1)
    
    data_url = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '.data.csv?date=latest&geography=' + codes_to_get + "&select=" + fields_we_want_query_str
    #print("Trying url {}".format(data_url))

    stream = session.get(data_url, stream=True).raw
    stream.decode_content = True
    df = pd.read_csv(stream)

    additionalURL = ''
    intRecordController = 0

    # The links below are limited to the first 25,000 cells per call.
    while True:
        start = datetime.datetime.now()
        additionalURL = '&RecordOffset=' + str(intRecordController)
        concatenatedURL = data_url + additionalURL
        stream = session.get(concatenatedURL, stream=True).raw
        stream.decode_content = True

        dataframe = pd.read_csv(stream, engine='c', na_filter=False)

        if (dataframe.empty):
            break

        # Additional code due to large datasets:
        # For a very basic progress monitor
        print("Just received record:" + (dataframe.tail(1).RECORD_OFFSET.to_string(index=False)) + ' of ' + (dataframe.tail(1).RECORD_COUNT.to_string(index=False)) + ' >>> ' + str(round((int(dataframe.tail(1).RECORD_OFFSET.to_string(index=False)) / int(dataframe.tail(1).RECORD_COUNT.to_string(index=False)) * 100),2)) + '%... in ' + str(datetime.datetime.now() - start))
        frames = [final_df, dataframe]
        final_df = pd.concat(frames)

        intRecordController = intRecordController + 25000
        
    # Going to write and flush every 1000 codes just in case
    if got_since_flush > 1000:
        df.to_csv("./out/output{}.csv".format(str(output_counter)), index=False)
        final_df = pd.DataFrame()
        got_since_flush = 0
        output_counter +=1


In [ ]:
final_df.to_csv('final_df.csv.gz', date_format='%Y%m%d', compression="gzip")

In [ ]:
print("Completion time for complete script is: ", datetime.datetime.now() - absolute_start)